In [ ]:
from dataclasses import dataclass
import torch
import torch.nn as nn
import inspect
from torch.nn import functional as F
import math

In [ ]:
class CausalSelfAttention(nn.Module):

  def __init__(self,config):
    super().__init__()
    assert config.n_embd % config.n_head == 0
    self.c_attn=nn.Linear(config.n_embd,3*config.n_embd)
    self.c_proj=nn.Linear(config.n_embd,config.n_embd)
    self.c_proj.NANOGPT_SCALE_INIT=1
    self.n_head=config.n_head
    self.n_embd=config.n_embd
    # 因果矩阵掩码
    self.register_buffer("bias",torch.tril(torch.ones(config.block_size,config.block_size))
                  .view(1,1,config.block_size,config.block_size))

  def forward(self,x):
    B,T,C=x.size()
    # 生成Q,K,V
    qkv=self.c_attn(x) #（B,T,3*C）
    q,k,v=qkv.split(self.n_embd,dim=2) #(B,T,C)
    k=k.view(B,T,self.n_head,C//self.n_head).transpose(1,2) #（B,nh,T,hs）
    q=q.view(B,T,self.n_head,C//self.n_head).transpose(1,2) #（B,nh,T,hs）
    v=v.view(B,T,self.n_head,C//self.n_head).transpose(1,2) #（B,nh,T,hs）

    # 计算注意力分数
    # att=(q@k.transpose(-2,-1))*(1.0/math.sqrt(k.size(-1)))
    # 应用因果掩码
    # att=att.masked_fill(self.bias[:,:,:T,:T]==0,float('-inf'))
    # softmax + 加权求和
    # att=F.softmax(att, dim=-1)
    # y=att@v

    # Flash Attention
    y=F.scaled_dot_product_attention(q,k,v,is_causal=True)
    # 重塑
    y=y.transpose(1,2).contiguous().view(B,T,C)
    y=self.c_proj(y)
    return y

class MLP(nn.Module):

  def __init__(self,config):
    super().__init__()
    self.c_fc=nn.Linear(config.n_embd,4*config.n_embd)
    self.gelu=nn.GELU(approximate='tanh')
    self.c_proj=nn.Linear(4*config.n_embd,config.n_embd)
    self.c_proj.NANOGPT_SCALE_INIT=1

  def forward(self,x):
    x=self.c_fc(x)
    x=self.gelu(x)
    x=self.c_proj(x)
    return x

class Block(nn.Module):

  def __init__(self,config):
    super().__init__()
    self.ln_1=nn.LayerNorm(config.n_embd)
    self.attn=CausalSelfAttention(config)
    self.ln_2=nn.LayerNorm(config.n_embd)
    self.mlp=MLP(config)

  def forward(self,x):
    x=x+self.attn(self.ln_1(x))
    x=x+self.mlp(self.ln_2(x))
    return x

@dataclass
class GPTConfig:
  block_size: int = 1024
  vocab_size: int = 50257
  n_layer: int = 12
  n_head: int = 12
  n_embd: int = 768

class GPT(nn.Module):

  def __init__(self,config):
    super().__init__()
    self.config=config

    self.transformer=nn.ModuleDict(dict(
        wte=nn.Embedding(config.vocab_size,config.n_embd),
        wpe=nn.Embedding(config.block_size,config.n_embd),
        h=nn.ModuleList(Block(config) for _ in range(config.n_layer)),
        ln_f=nn.LayerNorm(config.n_embd),
    ))
    self.lm_head=nn.Linear(config.n_embd,config.vocab_size,bias=False)

    # weight sharing 模式
    self.transformer.wte.weight=self.lm_head.weight

    self.apply(self._init_weights)

  def _init_weights(self,module):
    if isinstance(module,nn.Linear):
      std=0.02
      if hasattr(module,'NANOGPT_SCALE_INIT'):
        std*=(2*self.config.n_layer)**-0.5
      torch.nn.init.normal_(module.weight,mean=0.0,std=std)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module,nn.Embedding):
      torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)


  def forward(self,idx,targets=None):
    B,T=idx.size()
    assert T <= self.config.block_size,f"Cannot forward sequence of length{T},block size is {self.config.block_size}"
    # forward the token and position embeddings
    pos=torch.arange(0,T,dtype=torch.long,device=idx.device)
    pos_emb=self.transformer.wpe(pos)
    tok_emb=self.transformer.wte(idx)
    x=tok_emb+pos_emb
    # the blocks of the transformer
    for block in self.transformer.h:
      x=block(x)
    x=self.transformer.ln_f(x)
    logits=self.lm_head(x) #(B,T,vocab_size)
    loss=None
    if targets is not None:
      loss=F.cross_entropy(logits.view(-1,logits.size(-1)),targets.view(-1))
    return logits,loss

  @classmethod
  def from_pretrained(cls,model_type):
    assert model_type in {'gpt2','gpt2-medium','gpt2-large','gpt2-xl'}
    from transformers import GPT2LMHeadModel
    print("Loading weights from pretrained gpt: %s" % model_type)
    # 定义配置字典
    config_args={
        'gpt2':    dict(n_layer=12,n_head=12,n_embd=768), # 124M params
        'gpt2-medium': dict(n_layer=24,n_head=16,n_embd=1024),# 350M params
        'gpt2-large': dict(n_layer=36,n_head=20,n_embd=1280),# 774M params
        'gpt2-xl':  dict(n_layer=48,n_head=25,n_embd=1600),# 1558M params
    }[model_type]
    config_args['vocab_size']=50257
    config_args['block_size']=1024

    config=GPTConfig(**config_args)
    model=GPT(config)
    sd=model.state_dict()
    sd_keys=sd.keys()
    # 过滤掉 attention bias
    sd_keys=[k for k in sd_keys if not k.endswith('.attn.bias')]
    # 初始化 huggingface/transformers model
    model_hf=GPT2LMHeadModel.from_pretrained(model_type)
    sd_hf=model_hf.state_dict()

    # 权重映射和复制
    sd_keys_hf=sd_hf.keys()
    sd_keys_hf=[k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]
    sd_keys_hf=[k for k in sd_keys_hf if not k.endswith('.attn.bias')]
    transposed=['attn.c_attn.weight','attn.c_proj.weight','mlp.c_fc.weight','mlp.c_proj.weight']
    assert len(sd_keys)==len(sd_keys_hf),f"{len(sd_keys)} != {len(sd_keys_hf)}"
    for k in sd_keys_hf:
      if any(k.endswith(w) for w in transposed):
        assert sd_hf[k].shape[::-1]==sd[k].shape
        with torch.no_grad():
          sd[k].copy_(sd_hf[k].t())
      else:
        assert sd_hf[k].shape==sd[k].shape
        with torch.no_grad():
          sd[k].copy_(sd_hf[k])

    return model

  def configure_optimizers(self,weight_decay,learning_rate,device):
    # 获取模型所有参数
    param_dict={pn:p for pn,p in self.named_parameters()}
    # 筛选出带有梯度的参数
    param_dict={pn:p for pn,p in param_dict.items() if p.requires_grad}
    # 所有二维参数都会被衰减，其他不会
    decay_params=[p for n,p in param_dict.items() if p.dim()>=2]
    nodecay_params=[p for n,p in param_dict.items() if p.dim()<2]
    optim_groups=[
        {'params':decay_params,'weight_decay':weight_decay},
        {'params':nodecay_params,'weight_decay':0.0}
    ]
    num_decay_params=sum(p.numel() for p in decay_params)
    num_nodecay_params=sum(p.numel() for p in nodecay_params)
    print(f"num decayed tensors: {len(decay_params)},with {num_decay_params:,} params")
    print(f"num non-decayed tensors: {len(nodecay_params)},with {num_nodecay_params:,} params")

    # 使用Fused AdamW版本
    fused_available='fused' in inspect.signature(torch.optim.AdamW).parameters
    use_fused=fused_available and 'cuda' in device
    print(f"using fused AdamW:{use_fused}")
    optimizer=torch.optim.AdamW(optim_groups,lr=learning_rate,betas=(0.9,0.95),eps=1e-8,fused=use_fused)
    return optimizer


In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-16 02:27:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-09-16 02:27:13 (17.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import tiktoken

class DataLoaderLite:
  def __init__(self,B,T):
    self.B=B
    self.T=T

    with open('input.txt','r') as f:
      text=f.read()
    enc=tiktoken.get_encoding('gpt2')
    tokens=enc.encode(text)
    self.tokens=torch.tensor(tokens)
    print(f"loaded {len(self.tokens)} tokens")
    print(f"1 epoch = { len(self.tokens) // (B*T)} batches")

    self.current_position=0

  def next_batch(self):
    B,T=self.B,self.T
    buf=self.tokens[self.current_position:self.current_position+B*T+1]
    x=buf[:-1].view(B,T)
    y=buf[1:].view(B,T)

    self.current_position+=B*T
    # reset
    if self.current_position+(B*T+1) >= len(self.tokens):
      self.current_position=0
    return x,y

device="cpu"
if torch.cuda.is_available():
  device="cuda"
elif torch.backends.mps.is_available():
  device="mps"
print(f"Using device: {device}")

torch.manual_seed(1337)
if torch.cuda.is_available():
  torch.cuda.manual_seed(1337)

# gradient accumulation
total_batch_size=524288 # 2**19 ~0.5M
B=16
T=1024
assert total_batch_size % (B*T) == 0
grad_accum_steps=total_batch_size // (B*T)
print(f"grad_accum_steps = {grad_accum_steps}")

train_loader=DataLoaderLite(B=16,T=1024)

#-------------------------------------------------------------------------------
torch.set_float32_matmul_precision('high')

import time

# get logits
model=GPT(GPTConfig(vocab_size=50304))
model.to(device)
model=torch.compile(model)

# warmup && cosine decay
max_lr=6e-4
min_lr=max_lr*0.1
warmup_steps=10
max_steps=50
def get_lr(it):
  if it < warmup_steps:
    return max_lr*(it+1)/warmup_steps

  if it > max_steps:
    return min_lr

  decay_ratio=(it-warmup_steps)/(max_steps-warmup_steps)
  assert 0<=decay_ratio<=1
  coeff=0.5*(1.0+math.cos(math.pi*decay_ratio))
  return min_lr+(max_lr-min_lr)*coeff

# optimize
optimizer=model.configure_optimizers(weight_decay=0.1,learning_rate=6e-4,device=device)

for step in range(max_steps):
  t0=time.time()
  optimizer.zero_grad()
  loss_accum=0.0
  for micro_step in range(grad_accum_steps):
    x,y=train_loader.next_batch()
    x,y=x.to(device),y.to(device)
    with torch.autocast(device_type=device,dtype=torch.bfloat16):
      logits,loss=model(x,y)
    loss=loss/grad_accum_steps
    loss_accum+=loss.detach()
    loss.backward()
  norm=torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
  # determine and set the learning rate for this iteration
  lr=get_lr(step)
  for param_group in optimizer.param_groups:
    param_group['lr']=lr
  optimizer.step()
  # wait for GPU to finish work
  torch.cuda.synchronize()
  t1=time.time()
  dt=t1-t0
  tokens_processed=train_loader.B * train_loader.T * grad_accum_steps
  tokens_per_sec=tokens_processed / dt
  print(f"step {step:4d}|loss:{loss_accum.item():.6f}|lr:{lr:.4e}|norm:{norm:.4f}|dt:{dt*1000:.2f}ms|tok/sec:{tokens_per_sec:.2f}")


import sys;sys.exit(0)



Using device: cuda
grad_accum_steps = 32
loaded 338025 tokens
1 epoch = 20 batches
num decayed tensors: 50,with 124,354,560 params
num non-decayed tensors: 98,with 121,344 params
using fused AdamW:True
step    0|loss:10.938565|lr:6.0000e-05|norm:27.0125|dt:27615.35ms|tok/sec:18985.39
step    1|loss:9.649336|lr:1.2000e-04|norm:9.5176|dt:2772.30ms|tok/sec:189116.67
step    2|loss:9.225581|lr:1.8000e-04|norm:5.7287|dt:2778.90ms|tok/sec:188667.21
step    3|loss:9.813110|lr:2.4000e-04|norm:8.2066|dt:2775.81ms|tok/sec:188877.45
step    4|loss:9.191634|lr:3.0000e-04|norm:4.2995|dt:2773.14ms|tok/sec:189059.26
step    5|loss:8.678026|lr:3.6000e-04|norm:3.6285|dt:2778.27ms|tok/sec:188710.13
step    6|loss:8.294989|lr:4.2000e-04|norm:1.9536|dt:2780.59ms|tok/sec:188552.87
step    7|loss:8.068013|lr:4.8000e-04|norm:2.8517|dt:2778.75ms|tok/sec:188677.60
step    8|loss:7.714189|lr:5.4000e-04|norm:1.9113|dt:2776.39ms|tok/sec:188837.97
step    9|loss:7.347027|lr:6.0000e-04|norm:1.8001|dt:2782.65ms|tok/

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
